In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pingouin as pg
import cv2
from PIL import Image as Image_PIL
from os import listdir
from matplotlib.image import imread
import time
from random import sample
from sklearn import manifold, decomposition, preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import make_pipeline


from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

D:\Users\Damien\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


### Dataframes

In [2]:
df = pd.read_csv(
    r"C:\Users\Damien\Desktop\Data Scientist\P6\Dataset\df_clean.csv")

#### Préparation des données

**NLP**

In [3]:
train, test = train_test_split(df, test_size=0.25, random_state=42)

In [4]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(train['cat_lvl_1'])
y_test = encoder.fit_transform(test['cat_lvl_1'])

**Features**

In [5]:
pipe_svd_tfidf = make_pipeline(CountVectorizer(),
                               TfidfTransformer(),
                               TruncatedSVD(n_components=300))
pipe_svd_tfidf.fit(train['desc_clean'])
feat_train_svd_tfidf = pipe_svd_tfidf.transform(train['desc_clean'])
feat_test_svd_tfidf = pipe_svd_tfidf.transform(test['desc_clean'])

**IMG**

In [6]:
def get_files(path_list, size):
    liste_image = []
    compteur = 0
    for image in path_list:
        im = Image_PIL.open(image).resize((size, size))
        im2 = np.array(im)/255
        liste_image.append(im2)
        compteur += 1

    del compteur
    del im
    del im2

    return liste_image

In [7]:
path = "C:/Users/Damien/Desktop/Data Scientist/P6/Dataset/Flipkart/Images512/"

In [8]:
train['image'] = train['image'].apply(lambda x: path+str(x))
test['image'] = test['image'].apply(lambda x: path+str(x))

<ipython-input-8-0ff9aa097a5a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['image'] = train['image'].apply(lambda x: path+str(x))
<ipython-input-8-0ff9aa097a5a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['image'] = test['image'].apply(lambda x: path+str(x))


In [9]:
X_train_IMG = get_files(train['image'], size=224)
X_test_IMG = get_files(test['image'], size=224)

In [10]:
X_train_IMG = tf.stack(X_train_IMG)
y_train_IMG = tf.stack(y_train)

X_test_IMG = tf.stack(X_test_IMG)
y_test_IMG = tf.stack(y_test)

**Features**

In [11]:
from keras.applications.vgg16 import VGG16

In [12]:
# Charger VGG16 sans couches de classification (include_top=False)
VGG_model = VGG16(weights='imagenet', include_top=False,
                  input_shape=(224, 224, 3))

In [13]:
for layer in VGG_model.layers:
    layer.trainable = False

In [14]:
features_IMG_train = VGG_model.predict(X_train_IMG)

In [15]:
features_img_train = features_IMG_train.reshape(
    features_IMG_train.shape[0], -1)

In [16]:
features_IMG_test = VGG_model.predict(X_test_IMG)

In [17]:
features_img_test = features_IMG_test.reshape(features_IMG_test.shape[0], -1)

#### Modèles

**NLP**

In [18]:
clf_nlp = LogisticRegression()
clf_nlp.fit(feat_train_svd_tfidf, y_train)

LogisticRegression()

**IMG**

In [19]:
from sklearn.ensemble import RandomForestClassifier
clf_img = RandomForestClassifier()
clf_img.fit(features_img_train, y_train_IMG)

RandomForestClassifier()

#### Voting Classifier "à la main"

In [21]:
X_train_list = [features_img_train, feat_train_svd_tfidf]
X_test_list = [features_img_test, feat_test_svd_tfidf]

In [22]:
def fit_multiple_estimators(classifiers, X_list, y, sample_weights=None):

    # Fit all estimators with their respective feature arrays
    estimators_ = [clf.fit(X, y) if sample_weights is None else clf.fit(
        X, y, sample_weights) for clf, X in zip([clf for _, clf in classifiers], X_list)]

    return estimators_  # , le_


def predict_from_multiple_estimator(estimators, label_encoder, X_list, weights=None):

    # Predict 'soft' voting with probabilities

    pred1 = [clf.predict_proba(X) for clf, X in zip(estimators, X_list)]
    pred2 = np.average(pred1, axis=0, weights=weights)
    pred = np.argmax(pred2, axis=1)

    # Convert integer predictions to original labels:
    return label_encoder.inverse_transform(pred)

In [23]:
# Make sure the number of estimators here are equal to number of different feature datas
classifiers = [('TL',  clf_img),
               ('TFIDF', clf_nlp)]

In [24]:
pred_img = clf_img.predict_proba(features_img_test)

In [25]:
pred_nlp = clf_nlp.predict_proba(feat_test_svd_tfidf)

In [26]:
pred_clf = [pred_img, pred_nlp]

In [27]:
pred_soft = np.average(pred_clf, axis=0, weights=None)

In [28]:
pred = np.argmax(pred_soft, axis=1)

In [29]:
pred

array([2, 5, 2, 1, 6, 0, 3, 2, 4, 4, 6, 4, 5, 1, 2, 2, 0, 2, 4, 4, 5, 2,
       0, 2, 6, 1, 3, 5, 1, 5, 5, 2, 2, 6, 3, 4, 6, 3, 3, 2, 4, 1, 5, 4,
       1, 5, 3, 2, 0, 4, 5, 3, 5, 4, 2, 0, 1, 5, 2, 4, 4, 3, 6, 5, 6, 0,
       0, 3, 4, 6, 0, 2, 2, 2, 6, 2, 4, 3, 2, 4, 3, 4, 2, 5, 2, 6, 4, 3,
       3, 3, 4, 4, 5, 3, 6, 2, 0, 6, 3, 0, 4, 6, 3, 4, 2, 6, 1, 5, 4, 2,
       5, 4, 4, 0, 4, 1, 2, 6, 6, 2, 2, 2, 3, 4, 5, 3, 1, 3, 0, 0, 5, 1,
       3, 3, 0, 5, 3, 1, 5, 2, 5, 0, 3, 4, 4, 6, 6, 1, 3, 1, 6, 1, 6, 3,
       2, 4, 6, 1, 3, 6, 6, 6, 0, 2, 3, 6, 6, 2, 6, 2, 6, 1, 4, 6, 0, 6,
       3, 3, 4, 3, 0, 5, 4, 3, 0, 1, 0, 6, 2, 4, 1, 6, 5, 4, 5, 2, 1, 5,
       2, 2, 1, 0, 4, 2, 1, 3, 6, 0, 0, 3, 6, 2, 2, 2, 4, 1, 0, 4, 3, 3,
       4, 3, 0, 4, 2, 3, 4, 0, 2, 1, 2, 2, 1, 2, 5, 4, 0, 6, 1, 4, 4, 4,
       4, 0, 3, 2, 3, 3, 4, 5, 0, 6, 5, 0, 3, 0, 5, 3, 1, 0, 0, 6, 6],
      dtype=int64)

In [30]:
from sklearn.metrics import accuracy_score

print('accuracy sur jeu train : ',
      accuracy_score(pred, y_test))

accuracy sur jeu train :  0.9505703422053232


#### Voting Classifier avec Pipeline

In [31]:
features_img_train.shape

(787, 25088)

In [32]:
features_img_test.shape

(263, 25088)

In [33]:
feat_train_svd_tfidf.shape

(787, 300)

In [34]:
feat_test_svd_tfidf.shape

(263, 300)

In [35]:
features_train = np.concatenate(
    (features_img_train, feat_train_svd_tfidf), axis=1)

In [36]:
features_test = np.concatenate(
    (features_img_test, feat_test_svd_tfidf), axis=1)

In [37]:
features_train.shape

(787, 25388)

In [38]:
features_test.shape

(263, 25388)

In [39]:
from sklearn.base import TransformerMixin, BaseEstimator
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier

######################
# custom transformer for sklearn pipeline


class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

In [40]:
# fit clf_img with f1
pipe1 = Pipeline([
    # selecting features 0 to 25088 (f1) to be used with RF (clf_img)
    ('col_extract', ColumnExtractor(cols=range(0, 25089))),
    ('clf', clf_img)
])

pipe1.fit(features_train, y_train)  # sanity check
pipe1.score(features_test, y_test)  # sanity check

0.7756653992395437

In [41]:
# fit clf_nlp with f2
pipe2 = Pipeline([
    # selecting features 25089 to 25388 (f2) to be used with LR (clf_nlp)
    ('col_extract', ColumnExtractor(cols=range(25089, 25388))),
    ('clf', clf_nlp)
])

pipe2.fit(features_train, y_train)  # sanity check
pipe2.score(features_test, y_test)  # sanity check

0.9467680608365019

In [42]:
# voting classifier where clf1 fitted with df1 and clf2 fitted with df2
eclf = VotingClassifier(estimators=[(
    'f1-clf_img', pipe1), ('f2-clf_nlp', pipe2)], voting='soft', weights=[1, 1])
eclf.fit(features_train, y_train)
eclf.score(features_test, y_test)

0.9315589353612167

In [43]:
# ensemble/voting classifier where clf1 fitted with df1 and clf2 fitted with df2
eclf = VotingClassifier(estimators=[(
    'f1-clf_img', pipe1), ('f2-clf_nlp', pipe2)], voting='hard', weights=[1, 1])
eclf.fit(features_train, y_train)
eclf.score(features_test, y_test)

0.8136882129277566